<a href="https://colab.research.google.com/github/sergioshimura/3D-models/blob/main/Desafio_Alura_Google_Sergio_Shimura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Descrição do problema a resolver:**

Os candidatos às chamadas públicas, ao ler o edital, normalmente se deparam com dificuldades para encontrar informações importantes dentro da chamada, como por exemplo, datas importantes, documentos necessários, requisitos para o candidato, etc.
Este notebook visa facilitar a leitura de editais através da utilização de "Prompt" onde o usuário pode fazer perguntas relacionadas ao conteúdo de um ou vários documentos na forma PDF.

### **Author: Sergio Shimura**

Desafio Imersão Inteligência Artificial 2ª Edição (03 a 13/05/2024), Alura e Google

## **Descrição do código:**
opções do "pip install": -q (quiet)**
-U (update)

In [1]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai
!pip install -q -U pdfx

Configurações iniciais

In [2]:
from pathlib import Path
import hashlib
import google.generativeai as genai
import numpy as np
import pandas as pd
import pdfx

Para utilizar o API Key de forma segura, usar o código abaixo e habilitar, no menu do lado esquerdo, ticar o "Acesso ao notebook", nome = SECRET_KEY, Valor = "seu API KEY"

In [3]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key = api_key)

Hiperparametros do modelo: temperatura = nível de criatividade, top_p = probabilidade acumulada para os primeiros tokens, top_k = top k tokens

In [4]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

Configurações de segurança

In [5]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

Aqui é feita a carga do arquivo pdf que deve estar já disponível no menu do lado esquerdo, em arquivos - clicar em upload caso não apareça lá

In [6]:
pdf = pdfx.PDFx("document0.pdf")
pdf

Este comando transforma o PDF em texto

In [7]:
sample_text = pdf.get_text()
sample_text

'Ministério da Educação\nMinistério da Educação\nIns(cid:18)tuto Federal de Educação, Ciência e Tecnologia de São Paulo\nIns(cid:18)tuto Federal de Educação, Ciência e Tecnologia de São Paulo\nReitoria\nReitoria\nPRO-REITORIA PLAN. DESEN. INSTITUCIONAL\nPRO-REITORIA PLAN. DESEN. INSTITUCIONAL\n\n27 DE MARÇO DE 2024\n EDITAL Nº 9/2024 - PRO-PRD/RET/IFSP, DE  27 DE MARÇO DE 2024\n  EDITAL Nº 9/2024 - PRO-PRD/RET/IFSP, DE \n\n PROCESSO SELETIVO DE AFASTAMENTO REMUNERADO DOCENTE PARA PARTICIPAÇÃO EM PROGRAMA\nDE PÓS-GRADUAÇÃO STRICTO SENSU\n\nEDITAL PRO-PRD/CPPD nº09/2024\n\n2º SEMESTRE DE 2024\n\n \n\n \n\nA Pró-Reitoria de Planejamento e Desenvolvimento Institucional (PRO-PRD) e a Comissão Permanente de Pessoal Docente (CPPD) \ntorna pública a abertura das inscrições para o Processo Seletivo de docentes efetivos, no âmbito do Instituto Federal de Educação, Ciência \ne Tecnologia de São Paulo (IFSP), visando a classificação e preenchimento de vagas de Afastamento Remunerado Docente para \

O texto a ser cunsultado é colocado em "documents". **Caso hajam mais documentos, eles podem ser adicionados aqui bastando apenas acrescentar linhas de Titulo e Conteúdo para DOCUMENT2, DOCUMENT3, etc.**

In [8]:
DOCUMENT1 = {
    "Título": "Edital IFSP",
    "Conteúdo": sample_text
}
documents = [DOCUMENT1]

Aqui o texto é guardado em um dataframe para processamento posterior

In [9]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Edital IFSP,Ministério da Educação\nMinistério da Educação...


Modelo de embedding utilizado

In [10]:
model = "models/embedding-001"

Definição da função utilizada para fazer o "embedding"

In [11]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

Instrução para geração do embedding do texto

In [12]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Edital IFSP,Ministério da Educação\nMinistério da Educação...,"[0.028579198, -0.025965186, -0.044124056, 0.03..."


Definição da função de busca caso hajam mais documentos

In [13]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

Aqui é feita a busca em qual documento está a informação desejada

In [14]:
consulta = "qual a data limite de submissao?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Ministério da Educação
Ministério da Educação
Ins(cid:18)tuto Federal de Educação, Ciência e Tecnologia de São Paulo
Ins(cid:18)tuto Federal de Educação, Ciência e Tecnologia de São Paulo
Reitoria
Reitoria
PRO-REITORIA PLAN. DESEN. INSTITUCIONAL
PRO-REITORIA PLAN. DESEN. INSTITUCIONAL

27 DE MARÇO DE 2024
 EDITAL Nº 9/2024 - PRO-PRD/RET/IFSP, DE  27 DE MARÇO DE 2024
  EDITAL Nº 9/2024 - PRO-PRD/RET/IFSP, DE 

 PROCESSO SELETIVO DE AFASTAMENTO REMUNERADO DOCENTE PARA PARTICIPAÇÃO EM PROGRAMA
DE PÓS-GRADUAÇÃO STRICTO SENSU

EDITAL PRO-PRD/CPPD nº09/2024

2º SEMESTRE DE 2024

 

 

A Pró-Reitoria de Planejamento e Desenvolvimento Institucional (PRO-PRD) e a Comissão Permanente de Pessoal Docente (CPPD) 
torna pública a abertura das inscrições para o Processo Seletivo de docentes efetivos, no âmbito do Instituto Federal de Educação, Ciência 
e Tecnologia de São Paulo (IFSP), visando a classificação e preenchimento de vagas de Afastamento Remunerado Docente para 
Participação em Programa de

Aqui é feita a busca dentro do documento encontrado sobre a data limite de submissão

In [15]:
prompt = f"qual a data limite de submissao: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

18 de abril de 2024


Encontrado o documento, podemos fazer a pergunta 2: quais os requisitos

In [16]:
prompt = f"quais os requisitos do candidato: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Requisitos do Candidato:**

* Ser servidor efetivo do Instituto Federal de Educação, Ciência e Tecnologia de São Paulo (IFSP).
* Ter avaliação de desempenho funcional "Regular" ou "Bom" no último período avaliativo.
* Não ter sofrido punição disciplinar nos últimos 5 (cinco) anos.
* Não estar cumprindo pena de suspensão ou estando em afastamento do cargo.
* Não ter excedido o limite máximo de afastamento remunerado para pós-graduação previsto na Portaria Normativa RET IFSP nº 13/2021 (e sua alteração).
* Comprovar inscrição em programa de pós-graduação stricto sensu, reconhecido pelo Ministério da Educação (MEC), em nível de Mestrado, Doutorado ou Pós-Doutorado.
* Ter projeto de pesquisa ou atividades acadêmicas compatíveis com o programa de pós-graduação a ser cursado.


E pergunta 3: quais os documentos extras envolvidos

In [17]:
prompt = f"quais documentos envolvidos: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

- Edital Nº 9/2024 - PRO-PRD/RET/IFSP, de 27 de Março de 2024
- Portaria Normativa RET IFSP nº 12, de 07 de julho de 2021
- Portaria Normativas RET IFSP nº 13, de 07 de julho de 2021, e sua alteração (Portaria Normativa RET IFSP nº 69, de 21 de setembro de 2022)
- Portaria IFSP nº1869, de 26 de março de 2024 (indicativo de vagas)
